# Initial Data Exploration.

In [42]:
# --- Third-party libraries ---
import duckdb
import pandas as pd

In [43]:
# Initialize in-memory database and setup SQLite extension
con = duckdb.connect(database=':memory:')
con.execute("INSTALL sqlite;")
con.execute("LOAD sqlite;")

# Force DuckDB to treat all incoming data as strings (manual casting handled later)
con.execute("SET sqlite_all_varchar=true;")

# Attach the local database file as the 'edtech' schema
con.execute("ATTACH '../data/raw/enkeleksamen_case.db' AS edtech (TYPE SQLITE);")

print("DuckDB Connection Successful!")

DuckDB Connection Successful!


In [17]:
# Check all tables
tablas = con.sql("SHOW TABLES FROM edtech").df()
print("Tablesd:\n", tablas)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Tablesd:
                      name
0               _metadata
1       course_categories
2       course_progresses
3                 courses
4       lesson_progresses
5                 lessons
6          master_lessons
7                payments
8   product_viewed_events
9            quiz_answers
10  quiz_question_answers
11         quiz_questions
12                quizzes
13                ratings
14       signed_in_events
15        specialisations
16               students
17          subscriptions
18                 themes
19                 topics
20           universities
21                 videos


## Ground Truth
The goal of this initial phase is to define our Target Variable (Ground Truth).

Since we lack actual exam grades, we rely on the core business hypothesis: repurchasing a course strongly indicates a failed exam. Our first step is to explore the subscriptions table to flag students with multiple purchases for the same course.

However, relying solely on this rule introduces a severe bias. A student who buys a course but never interacts with it (0% progress), only to repurchase it later, represents a drop-out or postponement, not a learning failure. To prevent this, we cross-reference our data with the course_progresses table. By filtering out students with zero progress on their first attempt, we ensure our model learns exclusively from users who actually tried to study but still failed. This crucial quality filter drastically reduces false positives in our predictions.

In [ ]:
# Tables colums - suscription table
columnas_subs = con.sql("DESCRIBE edtech.subscriptions").df()
print("\nColumns in  'subscriptions':\n", columnas_subs[['column_name', 'column_type']])


Columnas en 'subscriptions':
     column_name column_type
0            id      BIGINT
1    student_id      BIGINT
2     course_id      BIGINT
3    payment_id      BIGINT
4          tier     VARCHAR
5  amount_cents      BIGINT
6          from     VARCHAR
7            to     VARCHAR
8    created_at     VARCHAR
9    updated_at     VARCHAR


In [ ]:
# colunm progress courses - table 
columnas_prog = con.sql("DESCRIBE edtech.course_progresses").df()
print("\nColumn in 'course_progresses':\n", columnas_prog[['column_name', 
                                                             'column_type']])


Columnas en 'course_progresses':
        column_name column_type
0        course_id      BIGINT
1       student_id      BIGINT
2  course_progress      BIGINT


In [ ]:
# Inspect 'courses' (To find the exam date)
columnas_courses = con.sql("DESCRIBE edtech.courses").df()
print("\n--- Columns in 'courses' ---")
print(columnas_courses[['column_name', 'column_type']])


--- Columns in 'courses' ---
             column_name column_type
0                     id      BIGINT
1                  title     VARCHAR
2     course_category_id      BIGINT
3              exam_date     VARCHAR
4     silver_price_cents      BIGINT
5       gold_price_cents      BIGINT
6    diamond_price_cents      BIGINT
7          lessons_count      BIGINT
8   quiz_questions_count      BIGINT
9               duration      BIGINT
10                active      BIGINT
11           purchasable      BIGINT
12                status     VARCHAR
13            created_at     VARCHAR


Before defining our Target Variable, we need to inspect the exact schema and data types
 of our core tables: subscriptions, course_progresses, and courses.

In [30]:
# Peek at a real record to understand the data format
ejemplo_subs = con.sql("SELECT * FROM edtech.subscriptions LIMIT 1").df()
print("\n--- Sample record from 'subscriptions' ---")
print(ejemplo_subs)


--- Sample record from 'subscriptions' ---
       id student_id course_id payment_id    tier amount_cents  \
0  150063      35422        50     182126  silver        69900   

                         from                          to  \
0  2023-01-01 08:58:42.997865  2022-12-31 09:28:22.154166   

                   created_at                  updated_at  
0  2023-01-01 08:58:42.999408  2023-07-03 21:01:05.635181  


In [31]:
print("\n--- Sample record: 'course_progresses' ---")
print(con.sql("SELECT * FROM edtech.course_progresses LIMIT 1").df())

print("\n--- Sample record: 'courses' ---")
print(con.sql("SELECT * FROM edtech.courses LIMIT 1").df())


--- Sample record: 'course_progresses' ---
  course_id student_id course_progress
0       448      98047              32

--- Sample record: 'courses' ---
  id                         title course_category_id   exam_date  \
0  1  Mikroøkonomi - Eldre versjon                 11  2022-12-31   

  silver_price_cents gold_price_cents diamond_price_cents lessons_count  \
0              99900           119800              219700            60   

  quiz_questions_count duration active purchasable status  \
0                   70        0  False       False   None   

                   created_at  
0  2016-08-01 07:17:22.607308  


## Target Variable (Ground Truth) Definition

In [ ]:
# We use TRY_CAST to handle dirty data like "NaT" strings in dates
query_cohort = """
WITH RankedSubscriptions AS (
    -- 1. Sort purchases per student and course to find the first attempt
    SELECT 
        TRY_CAST(student_id AS BIGINT) AS student_id,
        TRY_CAST(course_id AS BIGINT) AS course_id,
        TRY_CAST(created_at AS TIMESTAMP) AS purchase_date,
        ROW_NUMBER() OVER(PARTITION BY student_id, course_id ORDER BY TRY_CAST(created_at AS TIMESTAMP) ASC) as attempt_num
    FROM edtech.subscriptions
),
FirstAttempts AS (
    -- 2. Isolate the very first purchase attempt
    SELECT student_id, course_id, purchase_date
    FROM RankedSubscriptions
    WHERE attempt_num = 1
),
Retakes AS (
    -- 3. Identify students who repurchased (Target = 1)
    SELECT DISTINCT student_id, course_id
    FROM RankedSubscriptions
    WHERE attempt_num > 1
)
-- 4. Join everything and calculate the intervention date
SELECT 
    f.student_id,
    f.course_id,
    f.purchase_date,
    TRY_CAST(c.exam_date AS TIMESTAMP) AS exam_date,
    
    -- The Point-in-Time limit: 14 days before the exam BCONDITION--------
    TRY_CAST(c.exam_date AS TIMESTAMP) - INTERVAL 14 DAY AS intervention_date,
    
    -- Target Label: 1 if repurchased, 0 if not ----- TARGETTTT----
    CASE WHEN r.student_id IS NOT NULL THEN 1 ELSE 0 END AS target_reprobado,
    
    -- Course progress for the quality filter
    TRY_CAST(cp.course_progress AS BIGINT) AS final_progress
    
FROM FirstAttempts f
LEFT JOIN Retakes r 
  ON f.student_id = r.student_id AND f.course_id = r.course_id
LEFT JOIN edtech.courses c 
  ON f.course_id = TRY_CAST(c.id AS BIGINT)
LEFT JOIN edtech.course_progresses cp 
  ON f.student_id = TRY_CAST(cp.student_id AS BIGINT) 
 AND f.course_id = TRY_CAST(cp.course_id AS BIGINT)
  
-- Quality Filter: Only keep students who actually studied (>0 progress) and have a valid exam date
WHERE TRY_CAST(cp.course_progress AS BIGINT) > 0 
  AND TRY_CAST(c.exam_date AS TIMESTAMP) IS NOT NULL;
"""

# Execute the query in DuckDB
df_cohort = con.sql(query_cohort).df()

print(f"Total Analytical Cohort Size: {len(df_cohort)} student-course pairs")
print("\nTarget Variable Distribution (0 = Passed, 1 = Failed/Repurchased):")
print(df_cohort['target_reprobado'].value_counts(normalize=True) * 100)

# Save the cohort to a parquet file for the next phase
df_cohort.to_parquet('../data/processed/cohort_target.parquet')
print("\nCohort successfully saved to data/processed/cohort_target.parquet!")

Total Analytical Cohort Size: 75709 student-course pairs

Target Variable Distribution (0 = Passed, 1 = Failed/Repurchased):
target_reprobado
0    88.153324
1    11.846676
Name: proportion, dtype: float64

Cohort successfully saved to data/processed/cohort_target.parquet!


## Feature Engineering (Behavioral Metrics)

Extract learning behaviors.

In [39]:
# Inspect 'lesson_progresses' (to measure the student's pace in terms of progress)
lesson_columns = con.sql("DESCRIBE edtech.lesson_progresses").df()
print("--- Columns in 'lesson_progresses' ---")
print(lesson_columns[['column_name', 'column_type']])
print("\n--- Sample Record ---")
print(con.sql("SELECT * FROM edtech.lesson_progresses LIMIT 1").df())

# Inspect 'quiz_question_answers' (to measure success/mastery)
quiz_columns = con.sql("DESCRIBE edtech.quiz_question_answers").df()
print("\n--- Columns in 'quiz_question_answers' ---")
print(quiz_columns[['column_name', 'column_type']])
print("\n--- Sample Record ---")
print(con.sql("SELECT * FROM edtech.quiz_question_answers LIMIT 1").df())

# Summary Examples
print("\nQuiz Example:")
print(con.sql("SELECT * FROM edtech.quiz_question_answers LIMIT 1").df())
print("\nLesson Example:")
print(con.sql("SELECT * FROM edtech.lesson_progresses LIMIT 1").df())

--- Columns in 'lesson_progresses' ---
  column_name column_type
0  student_id     VARCHAR
1   lesson_id     VARCHAR
2    progress     VARCHAR
3  created_at     VARCHAR
4  updated_at     VARCHAR

--- Sample Record ---
  student_id lesson_id progress                  created_at  \
0     101098      8771      100  2026-02-01 00:19:21.634224   

                   updated_at  
0  2026-02-01 00:19:21.634224  

--- Columns in 'quiz_question_answers' ---
        column_name column_type
0        student_id     VARCHAR
1  quiz_question_id     VARCHAR
2           attempt     VARCHAR
3           verdict     VARCHAR
4         lesson_id     VARCHAR
5        created_at     VARCHAR

--- Sample Record ---
  student_id quiz_question_id attempt  verdict lesson_id  \
0      97174            12149       0  correct      <NA>   

                   created_at  
0  2024-11-04 11:44:44.370647  

Quiz Example:
  student_id quiz_question_id attempt  verdict lesson_id  \
0      97174            12149       0  c

In [ ]:
# We query the Pandas DataFrame 'df_cohort' directly using DuckDB!
query_features = """
WITH LessonStats AS (
    -- 1. Calculate Pacing / Effort (Strictly before intervention date)
    SELECT 
        c.student_id,
        c.course_id,
        COUNT(DISTINCT lp.lesson_id) AS lessons_started_count,
        AVG(TRY_CAST(lp.progress AS DOUBLE)) AS avg_lesson_progress,
        MIN(TRY_CAST(lp.created_at AS TIMESTAMP)) AS first_lesson_date
    FROM df_cohort c
    JOIN edtech.lesson_progresses lp 
      ON c.student_id = TRY_CAST(lp.student_id AS BIGINT)
    WHERE TRY_CAST(lp.created_at AS TIMESTAMP) <= c.intervention_date
    GROUP BY c.student_id, c.course_id
),
QuizStats AS (
    -- 2. Calculate Mastery / Success (Strictly before intervention date)
    SELECT 
        c.student_id,
        c.course_id,
        COUNT(qa.quiz_question_id) AS total_quiz_attempts,
        
        -- % of correct answers overall
        AVG(CASE WHEN qa.verdict = 'correct' THEN 1.0 ELSE 0.0 END) AS overall_correct_rate,
        
        -- % of correct answers on the VERY FIRST attempt (True Mastery)
        AVG(CASE WHEN qa.attempt = '0' AND qa.verdict = 'correct' THEN 1.0 
                 WHEN qa.attempt = '0' AND qa.verdict != 'correct' THEN 0.0 
                 ELSE NULL END) AS first_attempt_correct_rate,
                 
        MIN(TRY_CAST(qa.created_at AS TIMESTAMP)) AS first_quiz_date
    FROM df_cohort c
    JOIN edtech.quiz_question_answers qa 
      ON c.student_id = TRY_CAST(qa.student_id AS BIGINT)
    WHERE TRY_CAST(qa.created_at AS TIMESTAMP) <= c.intervention_date
    GROUP BY c.student_id, c.course_id
)
-- 3. Combine everything into our final Machine Learning Feature Matrix
SELECT 
    c.student_id,
    c.course_id,
    c.target_reprobado,
    
    -- Replace NULLs with 0 for students who didn't interact with lessons/quizzes
    COALESCE(l.lessons_started_count, 0) AS lessons_started_count,
    COALESCE(l.avg_lesson_progress, 0.0) AS avg_lesson_progress,
    COALESCE(q.total_quiz_attempts, 0) AS total_quiz_attempts,
    COALESCE(q.overall_correct_rate, 0.0) AS overall_correct_rate,
    COALESCE(q.first_attempt_correct_rate, 0.0) AS first_attempt_correct_rate,
    
    -- 4. Calculate Procrastination: Days between first activity and the intervention date
    CASE 
        WHEN l.first_lesson_date IS NOT NULL AND q.first_quiz_date IS NOT NULL 
             THEN DATE_DIFF('day', LEAST(l.first_lesson_date, q.first_quiz_date), c.intervention_date)
        WHEN l.first_lesson_date IS NOT NULL 
             THEN DATE_DIFF('day', l.first_lesson_date, c.intervention_date)
        WHEN q.first_quiz_date IS NOT NULL 
             THEN DATE_DIFF('day', q.first_quiz_date, c.intervention_date)
        ELSE 0 
    END AS days_active_before_intervention

FROM df_cohort c  
LEFT JOIN LessonStats l 
  ON c.student_id = l.student_id AND c.course_id = l.course_id
LEFT JOIN QuizStats q 
  ON c.student_id = q.student_id AND c.course_id = q.course_id
"""

# Execute the complex aggregation
df_features = con.sql(query_features).df()

print("--- Feature Matrix Completed ---")
print(f"Total Rows: {len(df_features)}")
print("\nSample of our Machine Learning variables:")
print(df_features.head())

# Save the final matrix!
df_features.to_parquet('../data/processed/feature_matrix.parquet')
print("\nFeature Matrix saved to data/processed/feature_matrix.parquet!")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

--- Feature Matrix Completed ---
Total Rows: 75709

Sample of our Machine Learning variables:
   student_id  course_id  target_reprobado  lessons_started_count  \
0       96303        203                 0                    112   
1       99101        203                 0                      6   
2       96637        127                 0                    116   
3       96966        127                 0                    134   
4       97134        155                 0                    167   

   avg_lesson_progress  total_quiz_attempts  overall_correct_rate  \
0           100.000000                  169              1.000000   
1           100.000000                    5              1.000000   
2           100.000000                  182              1.000000   
3            99.253731                  207              1.000000   
4            98.508982                  284              0.961268   

   first_attempt_correct_rate  days_active_before_intervention  
0          